# Training & Testing 

I used this code to train and test various models on the dataset. Once optimal performance was achieved best model was downloaded for each modality

### Loading Preprocessed Data

In [ ]:
# Enter the file id 
!gdown --id 1XJLa-kXOT_cBORvOgc3WnEkLX-dH


In [ ]:
!unzip -q wdata.zip

In [ ]:
import pandas as pd

In the cell below select a type of modal from 
- Temp
- Resp
- EDA
- ECG 
- EMG 

To Train the model corrosponding to that modal

In [ ]:
modal = 'Resp'

In [ ]:
def filter(file_num:int) :

  path = "/content/wdata/Modalities/S{}_train.csv".format(file_num)
  df = pd.read_csv(path)
  df = df[[modal,'label']]

  return df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Time Window 
start = 0
end = 1024

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
data = pd.DataFrame()
labels = pd.DataFrame()

In [ ]:
subjects = [2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

Creating the Data Frame

In [ ]:
for i in subjects :
  df = filter(i)
  x = df[modal]
  y = df['label']

  data[i] = x
  labels[i] = y 

# Min Max Scalar 

Since every person may have varying level (absolute value) of a modality, thus I have used min max scalar to standardize them

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(data)

In [ ]:
label = labels.to_numpy()

### Data Frame Creation

In [ ]:
def get_data(start,end) :

  x = np.concatenate([X[:,i] for i in range(start, end)]) 
  y = np.concatenate([label[:,i] for i in range(start, end )])

  return x,y

In [ ]:
def create_data_df(x,y) :
  data_df = pd.DataFrame()
  data_df['ecg'] = x
  data_df['label'] = y 

  return data_df

# Generating Sequence of Time Windows

In [ ]:
def generate_sequences(df) :

  all_data = []
  y = np.array([])

  for i in range(3) :
    
    data = df[df['label'] == i ]
    stop = df[df['label'] == i ].shape[0]
    index = 1


    # time interval
    start = 0
    end = 1024

    sequences = []


    while end < stop :
      sequences.append(data['ecg'][start:end])
      start = end
      end += 1024

    sequences = np.array(sequences)
    ln = sequences.shape[0]
    print(ln)

    y = np.concatenate((y,np.ones(ln)*i))

    if i == 0 :
      all_data = sequences
    else :
      all_data = np.vstack((all_data,sequences))


  return all_data, y

In [ ]:
def create_data(start,end) :
  x,y = get_data(start,end)
  df = create_data_df(x,y)
  all_data, y_data = generate_sequences(df)

  all_data = all_data.reshape((all_data.shape[0], all_data.shape[1], 1))

  return all_data,y_data

In [ ]:
import numpy as np

# Training Testing Data

In [ ]:
X_train, y_train = create_data(0,6)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(5) :
  plt.plot(X_train[i])

In [ ]:
X_val, y_val = create_data(6,9)
X_val.shape, y_val.shape

In [ ]:
num_classes = len(np.unique(y_train))

In [ ]:
num_classes

#### Shuffling Train data

In [ ]:
idx = np.random.permutation(len(X_train))
X_train = X_train[idx]
y_train = y_train[idx]

In [ ]:
from tensorflow import keras

# Model 

## Architecture

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    norml = keras.layers.BatchNormalization()(input_layer)


    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(norml)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    drop1 = keras.layers.Dropout(.2)(conv1)


    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(drop1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    drop2 = keras.layers.Dropout(.2)(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(drop2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    drop3 = keras.layers.Dropout(.2)(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(drop3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])
# keras.utils.plot_model(model, show_shapes=True)


## Callbacks & LR Schedular for better performance

In [ ]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data = (X_val,y_val),
    verbose=1,
)


Loading the best model

In [ ]:
model = keras.models.load_model("best_model.h5")

In [ ]:
x_test, y_test = create_data(9,15)
x_test.shape, y_test.shape

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()
